In [27]:
import pandas as pd
import torch
import numpy as np
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda'
device = torch.device(device)
print(f'device: {device}')

import optuna 

torch.__version__, optuna.__version__, 

device: cuda


('2.0.0+cu117', '3.1.1')

### Example 1

In [28]:

def objective(trial):
    x=trial.suggest_float("x", -10, 10)
    return(x-2)**2

study = optuna.create_study()
study.optimize(objective, n_trials=10)

[I 2023-04-21 09:59:38,748] A new study created in memory with name: no-name-bee14072-593f-4072-a1d2-41282f128ffe
[I 2023-04-21 09:59:38,750] Trial 0 finished with value: 52.76917558541202 and parameters: {'x': -5.264239504959347}. Best is trial 0 with value: 52.76917558541202.
[I 2023-04-21 09:59:38,750] Trial 1 finished with value: 0.09069408736362942 and parameters: {'x': 1.6988454095258891}. Best is trial 1 with value: 0.09069408736362942.
[I 2023-04-21 09:59:38,751] Trial 2 finished with value: 91.12293333897577 and parameters: {'x': -7.54583329725466}. Best is trial 1 with value: 0.09069408736362942.
[I 2023-04-21 09:59:38,751] Trial 3 finished with value: 96.34381382715746 and parameters: {'x': -7.815488466049841}. Best is trial 1 with value: 0.09069408736362942.
[I 2023-04-21 09:59:38,751] Trial 4 finished with value: 102.58353226376367 and parameters: {'x': -8.128352889970001}. Best is trial 1 with value: 0.09069408736362942.
[I 2023-04-21 09:59:38,752] Trial 5 finished with v

In [29]:
study.best_params

{'x': 1.6988454095258891}

## Example 2: Pytorch

In [30]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [31]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

[I 2023-04-21 09:59:38,938] A new study created in memory with name: no-name-97431e82-07b6-4f77-835c-bf590b7bc79f
[I 2023-04-21 09:59:40,325] Trial 0 finished with value: 0.1046875 and parameters: {'n_layers': 1, 'n_units_l0': 6, 'dropout_l0': 0.48239474600407456, 'optimizer': 'RMSprop', 'lr': 0.08176711629463074}. Best is trial 0 with value: 0.1046875.
[I 2023-04-21 09:59:41,829] Trial 1 finished with value: 0.0890625 and parameters: {'n_layers': 3, 'n_units_l0': 55, 'dropout_l0': 0.20138097895267937, 'n_units_l1': 22, 'dropout_l1': 0.31840509909574577, 'n_units_l2': 99, 'dropout_l2': 0.44775201306546475, 'optimizer': 'RMSprop', 'lr': 0.02257857188110478}. Best is trial 0 with value: 0.1046875.
[I 2023-04-21 09:59:43,263] Trial 2 finished with value: 0.7640625 and parameters: {'n_layers': 2, 'n_units_l0': 26, 'dropout_l0': 0.4182474431181499, 'n_units_l1': 39, 'dropout_l1': 0.21643871827422392, 'optimizer': 'RMSprop', 'lr': 0.0031521974811311733}. Best is trial 2 with value: 0.7640625

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  55
  Number of complete trials:  45
Best trial:
  Value:  0.8484375
  Params: 
    n_layers: 1
    n_units_l0: 110
    dropout_l0: 0.2377032125548892
    optimizer: Adam
    lr: 0.0036065782838494835


## Multi-objective example

In [1]:
import pandas as pd
import torch
import numpy as np

import torchvision
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda'
device = torch.device(device)
print(f'device: {device}')

import optuna 
print(
    f'optuna version: {optuna.__version__}',
    f'torch version: {torch.__version__}')

DEVICE = device
DIR = ".."
BATCHSIZE = 128
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


device: cuda
optuna version: 3.1.1 torch version: 2.0.0+cu117


In [3]:


def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features

    layers.append(nn.Linear(in_features, 10))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


# Defines training and evaluation.
def train_model(model, optimizer, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        F.nll_loss(model(data), target).backward()
        optimizer.step()


def eval_model(model, valid_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)
            pred = model(data).argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / N_VALID_EXAMPLES

    flops = FlopCountAnalysis(model, inputs=(torch.randn(1, 28 * 28).to(DEVICE),)).total()
    return flops, accuracy


def objective(trial):
    train_dataset = torchvision.datasets.FashionMNIST(
        DIR, train=True, download=True, transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_dataset, list(range(N_TRAIN_EXAMPLES))),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    val_dataset = torchvision.datasets.FashionMNIST(
        DIR, train=False, transform=torchvision.transforms.ToTensor()
    )
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(val_dataset, list(range(N_VALID_EXAMPLES))),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    model = define_model(trial).to(DEVICE)

    optimizer = torch.optim.Adam(
        model.parameters(), trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    )

    for epoch in range(10):
        train_model(model, optimizer, train_loader)
    flops, accuracy = eval_model(model, val_loader)
    return flops, accuracy

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from fvcore.nn import FlopCountAnalysis

import optuna

study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(objective, n_trials=30, timeout=300)



[I 2023-04-21 10:05:16,338] A new study created in memory with name: no-name-6730eaf6-8b97-449d-a393-2003b4d39910
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-04-21 10:05:18,378] Trial 0 finished with values: [61932.0, 0.57578125] and parameters: {'n_layers': 1, 'n_units_l0': 78, 'dropout_0': 0.2526197351991986, 'lr': 2.0120591778279165e-05}. 
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-04-21 10:05:19,512] Trial 1 finished with values: [8084.0, 0.33828125] and parameters: {'n_layers': 3, 'n_units_l0': 7, 'dropout_0': 0.3785759591533602, 'n_units_l1': 33, 'dropout_1': 0.4955383842440339, 'n_units_l2': 55, 'dropout_2': 0.48568359261706273, 'lr': 0.00019063266471821508}. 
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-04-21 10:05:20,623] Trial 2 finished with values: [64056.0, 0.68828125] and parameters: {'n_layers': 2, 'n_units_l0': 76, 'dropout_0': 0.30471176138592204, 'n_units_l1': 52, 'dropout_1': 0.253514012251650

In [6]:
optuna.visualization.plot_pareto_front(study, target_names=["FLOPS", "accuracy"])

In [19]:
import optuna
from optuna.study._study_direction import StudyDirection

def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2

study = optuna.create_study(direction=StudyDirection.MINIMIZE)
study.optimize(objective, n_trials=100)

print("Best trial:", study.best_trial)


[I 2023-04-24 09:00:01,430] A new study created in memory with name: no-name-516861e4-98ad-46b6-862c-0186668f541a
[I 2023-04-24 09:00:01,431] Trial 0 finished with value: 3.9258965928585887 and parameters: {'x': 0.01861245768058062}. Best is trial 0 with value: 3.9258965928585887.
[I 2023-04-24 09:00:01,432] Trial 1 finished with value: 6.969680477213171 and parameters: {'x': -0.640015241852435}. Best is trial 0 with value: 3.9258965928585887.
[I 2023-04-24 09:00:01,432] Trial 2 finished with value: 23.77498854674955 and parameters: {'x': 6.87596026919309}. Best is trial 0 with value: 3.9258965928585887.
[I 2023-04-24 09:00:01,433] Trial 3 finished with value: 31.31738504910754 and parameters: {'x': 7.596193800174145}. Best is trial 0 with value: 3.9258965928585887.
[I 2023-04-24 09:00:01,433] Trial 4 finished with value: 110.54534575711025 and parameters: {'x': -8.514054677293164}. Best is trial 0 with value: 3.9258965928585887.
[I 2023-04-24 09:00:01,434] Trial 5 finished with value:

Best trial: FrozenTrial(number=91, state=TrialState.COMPLETE, values=[0.0005744636644334113], datetime_start=datetime.datetime(2023, 4, 24, 9, 0, 1, 635493), datetime_complete=datetime.datetime(2023, 4, 24, 9, 0, 1, 637366), params={'x': 1.9760320283621369}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'x': FloatDistribution(high=10.0, log=False, low=-10.0, step=None)}, trial_id=91, value=None)


In [2]:
from optuna.study._multi_objective import _get_pareto_front_trials_2d

In [3]:
from collections import namedtuple
from optuna.trial import TrialState
from optuna.study import StudyDirection
from typing import Sequence, List

# Create a simple Trial object with just 'number', 'state', and 'values' attributes
SimpleTrial = namedtuple("SimpleTrial", ["number", "state", "values"])

# Create some dummy SimpleTrial objects
trial1 = SimpleTrial(number=1, state=TrialState.COMPLETE, values=[3, 7])
trial2 = SimpleTrial(number=2, state=TrialState.COMPLETE, values=[5, 6])
trial3 = SimpleTrial(number=3, state=TrialState.COMPLETE, values=[2, 8])
trial4 = SimpleTrial(number=4, state=TrialState.COMPLETE, values=[4, 9])

trials: Sequence[SimpleTrial] = [trial1, trial2, trial3, trial4]

# Define optimization directions for the two objectives
directions: Sequence[StudyDirection] = [StudyDirection.MINIMIZE, StudyDirection.MAXIMIZE]


In [5]:
pareto_front_trials = _get_pareto_front_trials_2d(trials, directions)


pareto_front_trials

[SimpleTrial(number=3, state=TrialState.COMPLETE, values=[2, 8]),
 SimpleTrial(number=4, state=TrialState.COMPLETE, values=[4, 9])]

In [12]:
import optuna
import inspect
print(inspect.getsource(optuna.visualization.plot_pareto_front))
#inspect.getsource(optuna.visualization.plot_pareto_front)


def plot_pareto_front(
    study: Study,
    *,
    target_names: Optional[List[str]] = None,
    include_dominated_trials: bool = True,
    axis_order: Optional[List[int]] = None,
    constraints_func: Optional[Callable[[FrozenTrial], Sequence[float]]] = None,
    targets: Optional[Callable[[FrozenTrial], Sequence[float]]] = None,
) -> "go.Figure":
    """Plot the Pareto front of a study.

    .. seealso::
        Please refer to :ref:`multi_objective` for the tutorial of the Pareto front visualization.

    Example:

        The following code snippet shows how to plot the Pareto front of a study.

        .. plotly::

            import optuna


            def objective(trial):
                x = trial.suggest_float("x", 0, 5)
                y = trial.suggest_float("y", 0, 3)

                v0 = 4 * x ** 2 + 4 * y ** 2
                v1 = (x - 5) ** 2 + (y - 5) ** 2
                return v0, v1


            study = optuna.create_study(directions=["minimize", "minimize"])
   